In [1]:
%%capture
!pip install chemdataextractor --no-deps
!pip install llama-index
!pip install llama-index-embeddings-huggingface
!pip install accelerate --upgrade
!pip install transformers --upgrade
!pip install llama-index-vector-stores-neo4jvector
!pip install llama-index-graph-stores-neo4j
!pip install flask
!pip install flask-ngrok
!pip install pyngrok
!pip install flask_cors
!pip install llama-index-postprocessor-longllmlingua
!pip install llmlingua
!pip install llama-index-retrievers-bm25
!pip install PyMuPDF
!pip install llama-index-llms-openai


from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.retrievers import BaseRetriever,VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
import llama_index.llms.openai
from llama_index.core import PropertyGraphIndex
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
import openai
from openai import OpenAI
from collections import defaultdict
from llama_index.core import get_response_synthesizer
import os

from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
import llama_index.llms.openai
import nest_asyncio
from llama_index.retrievers.bm25 import BM25Retriever
import fitz
from llama_index.postprocessor.longllmlingua import LongLLMLinguaPostprocessor




In [3]:

nest_asyncio.apply()
directory_path = '/content/summary'
doc = SimpleDirectoryReader(directory_path)
documents = doc.load_data()


def extract_paragraphs_from_pdf(pdf_path):
    document = fitz.open(pdf_path)

    paragraphs = []
    current_paragraph = ""

    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text = page.get_text("text")
        lines = text.split('\n')

        for line in lines:
            if line.strip() == "":  # empty line
                if current_paragraph:
                    paragraphs.append(current_paragraph.strip())
                    current_paragraph = ""
            else:
                current_paragraph += " " + line

        if current_paragraph:  # Add any remaining text as a paragraph
            paragraphs.append(current_paragraph.strip())

    return paragraphs

#we also want to also reliabily extract tables, in the future.
#https://huggingface.co/docs/transformers/main/en/model_doc/table-transformer


print(documents)




In [7]:

os.environ['OPENAI_API_KEY'] = "sk-proj-0g00JscMOmtrlUizIUrLT3BlbkFJJaqUNNk4j7eSWGWLVApU"
Settings.embed_model = OpenAIEmbedding(model_name="text-embedding-ada-002")
Settings.llm =  llama_index.llms.openai.OpenAI(model="gpt-4o", temperature=0.1)
openai_embedding = VectorStoreIndex.from_documents(documents)
#embedding0: openai_embedding
#TD: also try finetune DRAGON



In [8]:
class CustomRetriever(BaseRetriever):
    def __init__(
        self,
        vector_retriever =VectorIndexRetriever,
        keyword_retriever = BM25Retriever,
        mode = 'AND',
    ):
        self._vector_retriever = vector_retriever
        self._keyword_retriever = keyword_retriever
        if mode not in ("AND", "OR"):
            raise ValueError("Invalid mode.")
        self._mode = mode
        super().__init__()

    def _retrieve(self, query_bundle, similarity_top_k = 5):
        vector_nodes = self._vector_retriever.retrieve(query_bundle)
        keyword_nodes = self._keyword_retriever.retrieve(query_bundle)
        vector_ids = {n.node.node_id for n in vector_nodes}
        keyword_ids = {n.node.node_id for n in keyword_nodes}
        combined_dict = {n.node.node_id: n for n in vector_nodes}
        combined_dict.update({n.node.node_id: n for n in keyword_nodes})
        if self._mode == "AND":
            retrieve_ids = vector_ids.intersection(keyword_ids)
        else:
            retrieve_ids = vector_ids.union(keyword_ids)
        retrieve_nodes = [combined_dict[rid] for rid in retrieve_ids]
        retrieve_nodes.sort(key=lambda node: node.get_score(), reverse=True)
        if len(retrieve_nodes) >= similarity_top_k:
            return retrieve_nodes
        else:
            return retrieve_nodes[0:similarity_top_k]

retriever_bm25 =  BM25Retriever.from_defaults(nodes=documents, similarity_top_k=5)
retriever_similarity = openai_embedding.as_retriever(similarity_top_k=5,sub_retrivers = [])
retriever_hybird = CustomRetriever()

#note: when writing hybird search retriver, we should explicitly define it instead of concatenating sub_retrivers to control
#      the behavior of those. this is also the reason why sub_retrivers = [] is specified.

#td1: find ways to change bm25 params, not described in docs.

response_synthesizer = get_response_synthesizer()
node_postprocessor = LongLLMLinguaPostprocessor(
    instruction_str="Given the context, please answer the final question",
    target_token=700,
    rank_method="longllmlingua",
    additional_compress_kwargs={"reorder_context": "sort"}
)
#TD: use the algo to make a reranker/compressor myself


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [17]:
client = OpenAI()

def get_gpt_response(prompt):

    completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "user", "content": prompt}]
                            , max_tokens=500)

    return completion.choices[0].message.content

def make_query_engine(retriever,node_postprocesssors,response_synthesizer):
    #return: a query engine
    return RetrieverQueryEngine(
        response_synthesizer =response_synthesizer,
        retriever=retriever,
        node_postprocessors = node_postprocesssors
        )

def get_answer_and_evidences(query,query_engine):
    #return: str * list
    # return anwer as well as a list of supporting evidence
    res = query_engine.query(query)
    evidences = [node.text for node in res.source_nodes]
    return res.response, evidences

def getResponse(prompt):
  #implement https://openreview.net/pdf?id=5ud0h8OXwD
  ref = []
  for docs in documents:
    txt = docs.text
    question = f'''as an expert in chemistry, answer is there an answer or rationale to this question in the context?\n
                  Q: {prompt}\n
                  A: {txt}\n
                  answer 1 or 0, 1 stands for yes 0 for no, do not give anything further\n
                '''
    completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "user", "content": question}]
                            , max_tokens=500)
    print(completion.choices[0].message.content )
    if completion.choices[0].message.content == "1":
      ref.append(txt)

  for i in range(len(ref)):
    question = f'''as an expert in chemistry, write a rationale to answer the question in the context, include all numerical details\n
                  Q:{prompt}\n
                  context:{ref[i]}'''
    completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "user", "content": question}]
                            , max_tokens=500)
    ref[i] = completion.choices[0].message.content
  context = ""

  for rationale in ref:
    context += rationale
    context += "\n\n"


  prompt = f'''as an expert in chemistry, anwer Q based on context \n\n
              Q:{prompt}\n
              context:{context}\n
              following the example, be concise and summerize your answer in few sentences:
              example Q: Any mutation studies involved? If yes, then why and what is the result?
              example A:
                        -	 Mutagenesis experiment where Arg90 was mutated to citrulline (Guimaraes et al). Not directly from this paper
                        -	The reason was to see the impact of Arg90 on the activation barrier of the reaction.
                        -	Result showed a lowering of the activation energy.
            '''

  completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "user", "content": prompt}]
                            , max_tokens=500)
  return completion.choices[0].message.content


def create_subquestion(prompt):
    ins = '''instruction: break down the following questions into a few intermediate questions, and lists all your intermediate questions in order, indexed by [1], [2], [3], ...\n\n
             if breakdown is not needed, simply return [1]:original question\n\n
example question: If Peter was to plan a two-day tour of Pittsburgh, what should be his plan?\n\n
example answer: [1]: how should he arrive at Pittsburgh? [2]: where should he live in Pittsburgh? [3]: what should he do on the first day? [4]: what should he do on the second day?\n\n
example question: that need no breakdown: what is the melting point of HCl\n\n
example answer: [1]: what is the melting point of HCl
question: '''
    prompt = ins + prompt
    response  = get_gpt_response(prompt)
    i = 1
    D = defaultdict(str)
    while True:
        start = response.find(f"[{i}]")
        end = response.find(f"[{i+1}]")
        if start == -1:
            return D
        D[i] = response[start:end]
        i+=1

def answer(query, query_engine):
  #implement: https://arxiv.org/pdf/2205.10625
  Q = create_subquestion(query)
  E = defaultdict(str)
  A = copy.deepcopy(Q)
  additional_context = "With the context of previous questions and answers: "
  for key in A.keys():
    res = query_engine.query(A[key] + additional_context)
    E[key] = [node.text for node in res.source_nodes]
    A[key] = res.response
    additional_context += f"Q: {Q[key]}"
    additional_context += f"A: {A[key]}"
  prompt = f"As an expert in chemistry, summerize the intermediate questions/answer described below to answer the question: {query} \n"
  for i in range(1,len(Q)):
    prompt += f"Q{i}: {Q[i]}"
    prompt += f"A{i}: {A[i]}\n"
  prompt += "you can assume the reader's expert background knowledge in chemistry"
  return get_gpt_response(prompt)



We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


('The level of theory used in the quantum mechanical (QM) study is local electron correlation with MPRO.', ['um Chemical Analysis of\nReaction Paths inase:ationals andstaticization\nBS SZEK,1FDERIK CLAEY2\nAN. MUL,2W. ANDRJ SOKALI1 ofical,roc\nze˙ia´2 forational,\n Kingdom\nReceived1500;1\n1 in Wiley).I/2135\n: We have a detailed, quantum chemical, decomposition\n the physical of key the modeleateaseCM), for several activeations by high level quantumchan). In to previousy , interactions between selectedues siteM wereed alongthe path, for several paths. The interaction energy is calculated toM order theory and intoicallycomponents (,,, and). Thisows, the dominant interaction is differentialization by Arg90: this residuesignantly the state) theate in all the paths.actions the site of are thestatic,as components, example electron are constant during.static effects are be forrier reactionhe. of four paths fromMMing thatiffer the are thestatic interactions\n Szefk b@.\npl or. Mul:@\n:RC,, I